In [7]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from lxml import etree
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from tqdm.notebook import tqdm
import time
import psycopg2
import json
import re
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Import file with (asin,product_url,brand,title,product_id)

In [8]:
df=pd.read_excel("manomano_to_kinshu.xlsx")
df

,sku,title,brand,product_url,product_id
0,100010651,Pet Food/Alimentation/Bol Tapis/Pad pour Chien...,FURWOO,https://www.manomano.fr/p/pet-foodalimentation...,70852813
1,100011085,"Jouet Chat 4 en 1, Balle Interactive Chat avec...",FURWOO,https://www.manomano.fr/p/jouet-chat-4-en-1-ba...,70853249
2,100011899,"Jouet Chat 4 en 1, Balle Interactive Chat avec...",FURWOO,https://www.manomano.fr/p/jouet-chat-4-en-1-ba...,70854055
3,100014006,Nourriture pour chaton PVC bol d'alimentation ...,FURWOO,https://www.manomano.fr/p/nourriture-pour-chat...,70856159
4,100015021,Tanaga Bronze 20 - Douche solaire basse de jar...,Franchini,https://www.manomano.fr/p/tanaga-20-doccia-sol...,70774416
...,...,...,...,...,...
6648,99843756,Seau à eau pliable avec couvercle verrouillabl...,CHENDYY,https://www.manomano.fr/p/seau-a-eau-pliable-a...,70773027
6649,99843819,Animal Domestique Chien Harnais Gilet réglable...,CHENDYY,https://www.manomano.fr/p/animal-domestique-ch...,70773104
6650,99932222,LightMe LM85263 LED CEE 2021 G (A - G) E14 for...,LIGHT ME,https://www.manomano.fr/p/lightme-lm85263-led-...,70815163
6651,99969798,"Fontaine a Eau Chat, Distributeur Eau pour Cha...",LINGHHANG,https://www.manomano.fr/p/fontaine-a-eau-chat-...,70829829


In [9]:
df.dropna(subset='sku',inplace=True)
df.drop_duplicates(subset='sku',inplace=True)
df.dropna(subset='product_url',inplace=True)
df.drop_duplicates(subset='product_url',inplace=True)
df.reset_index(drop=True, inplace=True)
df['categories']=''
df['first_image']=''
df.shape #this will give you number of rows and coloumns

(6652, 7)

In [10]:
s = Service("../chromedriver_win32/chromedriver.exe")

In [11]:
chrome_opt=Options()
chrome_opt.add_argument("--incognito")
driver = webdriver.Chrome(service=s,options=chrome_opt)
capabilities = driver.capabilities
chrome_driver_version = capabilities['chrome']['chromedriverVersion']

# Print the ChromeDriver version
print("ChromeDriver version:", chrome_driver_version)


ChromeDriver version: 114.0.5735.90 (386bc09e8f4f2e025eddae123f36f6263096ae49-refs/branch-heads/5735@{#1052})


In [18]:
for i in tqdm(range(0,5)):
    Url=df['product_url'][i]
    driver = webdriver.Chrome(service=s,options=chrome_opt)
    print('************************')
    print(i)
    print(Url)

    driver.get(Url)
    bs=BeautifulSoup(driver.page_source, "html.parser")
    
    try:
        aa=bs.find('ul',class_="mrmSZI ZhMtQ3 v5nnOB BGNvLC")
        bb=aa.find_all('li',class_='veEoYG')
        cat=[]
        for j in range(1,len(bb)-1):
            cc=bb[j].text
            cat.append(cc)
    
        print(cat)
        
        df['categories'][i]=cat
        
    except:
        print('category not found')
        
    try:
        images=bs.find('div',class_="c9uNnvv")
        first_image=images.find('img').get('src')
        print(first_image)
    except:
        print('image not found')
    driver.quit()
        

  0%|          | 0/5 [00:00<?, ?it/s]

************************
0
https://www.manomano.fr/p/pet-foodalimentationbol-tapispad-pour-chien-et-chat-gris-carre-40x60cm-1575x236in-70852813?product_id=100010651
['Animalerie', 'Chien', 'Couchage pour chien', 'Tapis, coussin et matelas pour chien']
https://cdn.manomano.com/images/images_products/31452114/P/100010651_1.jpg


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_150404\4138070775.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['categories'][i]=cat


************************
1
https://www.manomano.fr/p/jouet-chat-4-en-1-balle-interactive-chat-avec-double-boule-interactif-pour-chat-jouet-distributeur-croquette-chat-facile-a-nettoyer-jouet-interactif-chat-avec-baton-pour-chat-jaune-70853249?product_id=100011085
['Animalerie', 'Chat', 'Jouet et jeux pour chat', 'Jouet interactif pour chat']
https://cdn.manomano.com/images/images_products/31452114/P/100011085_1.jpg
************************
2
https://www.manomano.fr/p/jouet-chat-4-en-1-balle-interactive-chat-avec-double-boule-interactif-pour-chat-jouet-distributeur-croquette-chat-facile-a-nettoyer-jouet-interactif-chat-avec-baton-pour-chat-vert-70854055?product_id=100011899
['Animalerie', 'Chat', 'Jouet et jeux pour chat', 'Jouet interactif pour chat']
https://cdn.manomano.com/images/images_products/31452114/P/100011899_1.jpg
************************
3
https://www.manomano.fr/p/nourriture-pour-chaton-pvc-bol-dalimentation-napperon-antiderapant-tapis-de-litiere-pour-animaux-de-compagnie-

In [ ]:
df

In [19]:
request_no=input("Enter the request number of the requests seperated with underscore(_) ")
#547_552
file='final_output_manomano_france_{}.xlsx'.format(request_no)

Enter the request number of the requests seperated with underscore(_) 223


In [20]:
df.to_excel(file)

In [ ]:
email_user = 'alerts@1digitalstack.ai'
email_password = ''
email_send = ['nitin.bhayana@1digitalstack.ai','ghansham.pindoriya@1digitalstack.ai','karan.kumar@1digitalstack.ai','arihant.singh@1digitalstack.ai','sambhav.jain@1digitalstack.ai','ankur.dwivedi@1digitalstack.ai','kinshu@1digitalstack.ai']

subject = file
msg = MIMEMultipart()
msg['From'] = email_user
#msg['From'] = "'Abhishek' <{email_user}>"
msg['To'] = ', '.join(email_send)
msg['Subject'] = subject

body = 'Hi,\n\nPFA of today''s mapping request.Please reply as done as you pick this job.'
msg.attach(MIMEText(body,'plain'))

filename=[file]

for file in filename:
    attachment  =open(file,'rb')
    part = MIMEBase('application','octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition',"attachment; filename= "+file)
    msg.attach(part)
    text = msg.as_string()

server = smtplib.SMTP('1digitalstack.com',25)
#server.login(email_user,email_password)

server.sendmail(email_user,email_send,text)
server.quit()